## Les réseaux de neurones - Utilisation GPU

Google fournit un service cloud sur Jupyter Notebook, et permet d'utiliser leurs GPUs gratuitement. Tous les notebooks sont stockés dans Google Drive.

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn
import os
import sys
import time
from google.colab import drive
drive.mount('/content/gdrive')
import torch


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/TD_DL/ # Modifier le répertoire au besoin

[Errno 2] No such file or directory: '/content/gdrive/My Drive/Colab Notebooks/TD_DL/'
/content


### Chargement des données «MNIST»

In [0]:
repertoire_mnist = '../data/mnist/' # Modifier le répertoire au besoin

In [0]:
def charger_mnist(repertoire, etiquettes=None, max_par_etiquettes=None):
    if etiquettes is None:
         etiquettes = np.arange(10)
    images_list = [None] * len(etiquettes)
    labels_list = [None] * len(etiquettes)
    for i, val in enumerate(etiquettes):
        nom_fichier = repertoire + f'mnist_{val}.gz'
        images_list[i] = np.genfromtxt(nom_fichier, max_rows=max_par_etiquettes, dtype=np.float32)
        nb = images_list[i].shape[0]

        labels_list[i] = i*np.ones(nb, dtype=np.int64)
        print(val, ':', nb, 'images')
        
    x = np.vstack(images_list)
    y = np.concatenate(labels_list)
    print('Total :', len(y), 'images')
    return x, y

In [0]:
data_x, data_y = charger_mnist(repertoire_mnist, etiquettes=None, max_par_etiquettes=6000)
data_x = data_x / 255

0 : 1000 images
1 : 1000 images
2 : 1000 images
3 : 1000 images
4 : 1000 images
5 : 1000 images
6 : 1000 images
7 : 1000 images
8 : 1000 images
9 : 1000 images
Total : 10000 images


In [0]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.7, random_state=42)
print('train_x:', train_x.shape)
print('test_x:', test_x.shape)
print('train_y:', train_y.shape)
print('test_y:', test_y.shape)

train_x: (3000, 784)
test_x: (7000, 784)
train_y: (3000,)
test_y: (7000,)


In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_x)
train_x_prime = scaler.transform(train_x)
test_x_prime = scaler.transform(test_x)

### Créons un réseau de neurones
Nous réutilisons la classe `reseau_classif_generique` introduite lors du TD précédent.

In [0]:
class UneArchiPourMNIST(nn.Module):
    def __init__(self, nb_filtres_par_couche, taille_noyau_par_couche):
        # Initialisation de la classe de base nn.Module
        super().__init__()
        
        # Créons une couche de convolution 
        self.modele_conv = nn.Sequential(
            nn.Conv2d(1, nb_filtres_par_couche[0], kernel_size=taille_noyau_par_couche[0]),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(nb_filtres_par_couche[0], nb_filtres_par_couche[1], kernel_size=taille_noyau_par_couche[1]),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        # La convolution est suivie d'une couche de sortie
        nb_pixels = (28-taille_noyau_par_couche[0]+1)//2
        nb_pixels = (nb_pixels - taille_noyau_par_couche[1] + 1) // 2
        self.nb_neurones_du_milieu = nb_filtres_par_couche[1] * (nb_pixels)**2
        
        self.modele_plein = nn.Sequential(
            nn.Linear(self.nb_neurones_du_milieu, 10),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, x):
        # Propageons la «batch». Notez que nous devons redimensionner nos données consciencieusement
        x0 = x.view(-1, 1, 28, 28)
        x1 = self.modele_conv(x0)
        x2 = x1.view(-1, self.nb_neurones_du_milieu)
        x3 = self.modele_plein(x2)
        return x3

In [0]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from copy import deepcopy
import time


class ReseauClassifGenerique:
    """ Classe destinée en encapsuler une architecture de réseau de neurones pour la classification 
    multi-classe. L'apprentissage est effectué par descente de gradient stochastique avec «minibatch»,
    et il est possible de déterminer l'arrêt de l'optimisation par «early stopping».

    Paramètres
    ----------
    modele: Objet contenant l'architecture du réseau de neurones à optimiser. 
    eta, alpha: Parametres de la descente en gradient stochastique (taille du gradient et momentum).
    nb_epoques: Nombre d'époques maximum de la descente en gradient stochastique.
    taille_batch: Nombre d'exemples pour chaque «minibatch».
    fraction_validation: Fraction (entre 0.0 à 1.0) des exemples d'apprentissage à utiliser pour
                         créer un ensemble de validation pour le «early stopping».
                         Par défaut fraction_validation=None et il n'y a pas de «early stopping».
    patience: Paramètre de patience pour le «early stopping».
    seed: Germe du générateur de nombres aléatoires.
    """
    def __init__(self, modele, eta=0.4, alpha=0.1, nb_epoques=10, taille_batch=32, 
                 fraction_validation=0.2, patience=10, seed=None, device='cpu'):
        # Initialisation des paramètres
        self.modele = modele
        self.eta = eta
        self.alpha = alpha
        self.nb_epoques = nb_epoques
        self.taille_batch = taille_batch
        self.fraction_validation = fraction_validation
        self.patience = patience
        self.seed = seed
        self.device = device
        
        # Ces deux listes serviront à maintenir des statistiques lors de l'optimisation
        self.liste_objectif = list()
        self.liste_erreur_train = list()
        self.liste_erreur_valid = list()
        
    def _trace(self, obj, erreur_train, erreur_valid):
        self.liste_objectif.append(obj.item())    
        self.liste_erreur_train.append(erreur_train.item())        
        self.liste_erreur_valid.append(erreur_valid.item()) 
        
    def apprentissage(self, x, y): 
        if self.seed is not None:
            torch.manual_seed(self.seed)
            
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.int64) 
        nb_sorties = len(torch.unique(y))

        if self.fraction_validation is None:
            # Aucun «early stopping»
            early_stopping = False
            erreur_valid = None
            meilleure_epoque = None
            
            # Toutes les données sont dédiées à l'apprentissage
            train_data = TensorDataset(x, y)
            max_epoques = self.nb_epoques
        else:
            early_stopping = True
            
            # Création de l'ensemble de validation pour le «early stopping»
            nb_valid = int(self.fraction_validation * len(y))
            nb_train = len(y) - nb_valid
            
            train_data = TensorDataset(x[:nb_train], y[:nb_train])
            valid_data = TensorDataset(x[nb_train:], y[nb_train:])
            
            # Initialisation des variables pour le «early stopping»
            meilleure_erreur = 2.
            meilleure_epoque = 0
            max_epoques = self.patience
            
        # Initialisation du problème d'optimisation
        sampler = DataLoader(train_data, batch_size=self.taille_batch, shuffle=True) 
        perte_logistique = nn.NLLLoss()       
        optimizer = torch.optim.SGD(self.modele.parameters(), lr=self.eta, momentum=self.alpha)
           
        # Descente de gradient
        t = 0
        run_time = time.time()
        while t < min(max_epoques, self.nb_epoques):
            t += 1
            # Une époque correspond à un passage sur toutes les «mini-batch»
            liste_pertes = list()
            for batch_x, batch_y in sampler:
                batch_x, batch_y = batch_x.to(self.device), batch_y.to(self.device)
                
                # Propagation avant
                y_pred = self.modele(batch_x)
                perte = perte_logistique(y_pred, batch_y)

                # Rétropropagation
                optimizer.zero_grad()
                perte.backward()
                optimizer.step()
                
                liste_pertes.append(perte.item())
                
            # Pour fin de consultation future, on conserve les statistiques sur la fonction objectif
            perte_moyenne = np.mean(liste_pertes)
            message = f'[{t:3}] perte: {perte_moyenne:.5f}'
            
            # Calcule l'erreur sur l'ensemble d'entraînement
            with torch.no_grad():
                pred_train = self.modele(train_data.tensors[0].to(self.device))
                pred_train = torch.argmax(pred_train, dim=1)
                erreur_train = 1 - torch.mean(pred_train == train_data.tensors[1].to(self.device), dtype=torch.float32)
                message += f' | erreur train: {erreur_train:3f}'            
            
            if early_stopping:
                # Calcule l'erreur sur l'ensemble de validation
                with torch.no_grad():
                    pred_valid = self.modele(valid_data.tensors[0].to(self.device))
                    pred_valid = torch.argmax(pred_valid, dim=1)
                    erreur_valid = 1 - torch.mean(pred_valid == valid_data.tensors[1].to(self.device), dtype=torch.float32)
                    message += f' | erreur valid: {erreur_valid:3f}'
               
                if erreur_valid < meilleure_erreur:
                    # Conserve le meilleur modèle 
                    meilleur_modele = deepcopy(self.modele).to(self.device)
                    meilleure_erreur = erreur_valid
                    meilleure_epoque = t
                    max_epoques = t + self.patience
                    message += f' <-- meilleur modèle à ce jour (max_t={max_epoques})' 
            
            
            # Fin de l'époque: affiche le message d'état à l'utilisateur avant de passer à l'époque t+1 
            print(message)
            self._trace(perte_moyenne, erreur_train, erreur_valid)
        
            if t % 5 == 0:
                print("Le temps d'apprentissage: {}".format(time.time() - run_time))
          
        print('=== Optimisation terminée ===')
        print("Le temps d'apprentissage complet: {}".format(time.time() - run_time))
        
        # Dans le cas du «early stopping», on retourne à l'état du modèle offrant la meilleure précision en validation  
        if early_stopping:
            self.modele = meilleur_modele
            self.meilleure_epoque = meilleure_epoque
            print(f"Early stopping à l'époque #{meilleure_epoque}, avec erreur de validation de {meilleure_erreur}")
                
    def prediction(self, x):
        x = torch.tensor(x, dtype=torch.float32) # On s'assure que les données sont dans le bon format pytorch       
        
        with torch.no_grad():
            pred = self.modele(x.to(self.device))            # Propagation avant 
            pred = torch.argmax(pred, dim=1) # La prédiction correspond à l'indice du neurone de sortie ayant la valeure maximale
            pred = pred.cpu()
     
        return np.array(pred.detach()) # Retourne le résultat en format numpy


In [0]:
mon_archi = UneArchiPourMNIST(nb_filtres_par_couche=[32, 32], taille_noyau_par_couche=[3,3])

### Choisir le «device» (CPU ou GPU) sur lequel lancer l'apprentissage.

#### Apprentissage sur CPU.

In [0]:
use_cuda = False
# print(use_cuda)
print("CUDA Available: ", torch.cuda.is_available())
device = torch.device("cuda:0" if (use_cuda and torch.cuda.is_available()) else "cpu")
print(device)

CUDA Available:  True
cpu


In [0]:
mon_archi.to(device)

UneArchiPourMNIST(
  (modele_conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (modele_plein): Sequential(
    (0): Linear(in_features=800, out_features=10, bias=True)
    (1): LogSoftmax()
  )
)

In [0]:
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=30, taille_batch=32, seed=16, device=device)

In [0]:
R.apprentissage(train_x_prime, train_y)

[  1] perte: 0.78593 | erreur train: 0.097500 | erreur valid: 0.126667 <-- meilleur modèle à ce jour (max_t=11)
[  2] perte: 0.22526 | erreur train: 0.041667 | erreur valid: 0.068333 <-- meilleur modèle à ce jour (max_t=12)
[  3] perte: 0.14316 | erreur train: 0.026667 | erreur valid: 0.060000 <-- meilleur modèle à ce jour (max_t=13)
[  4] perte: 0.09176 | erreur train: 0.014583 | erreur valid: 0.055000 <-- meilleur modèle à ce jour (max_t=14)
[  5] perte: 0.06710 | erreur train: 0.010000 | erreur valid: 0.056667
Le temps d'apprentissage: 10.913705825805664
[  6] perte: 0.04659 | erreur train: 0.009167 | erreur valid: 0.050000 <-- meilleur modèle à ce jour (max_t=16)
[  7] perte: 0.03352 | erreur train: 0.007083 | erreur valid: 0.048333 <-- meilleur modèle à ce jour (max_t=17)
[  8] perte: 0.02607 | erreur train: 0.002500 | erreur valid: 0.053333
[  9] perte: 0.01784 | erreur train: 0.001250 | erreur valid: 0.048333
[ 10] perte: 0.00885 | erreur train: 0.000000 | erreur valid: 0.051667

In [0]:
from sklearn.metrics import accuracy_score
train_pred = R.prediction(train_x_prime)
test_pred = R.prediction(test_x_prime)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

Précision train: 0.9846666666666667
Précision test : 0.9532857142857143


#### Apprentissage sur GPUs.

In [0]:
use_cuda = True
print(use_cuda)
print("CUDA Available: ", torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
print(device)

True
CUDA Available:  True
cuda


In [0]:
mon_archi.to(device)

UneArchiPourMNIST(
  (modele_conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (modele_plein): Sequential(
    (0): Linear(in_features=800, out_features=10, bias=True)
    (1): LogSoftmax()
  )
)

In [0]:
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=20, taille_batch=32, seed=16, device=device)

In [0]:
R.apprentissage(train_x_prime, train_y)

[  1] perte: 0.00187 | erreur train: 0.000000 | erreur valid: 0.050000 <-- meilleur modèle à ce jour (max_t=11)
[  2] perte: 0.00158 | erreur train: 0.000000 | erreur valid: 0.051667
[  3] perte: 0.00144 | erreur train: 0.000000 | erreur valid: 0.050000
[  4] perte: 0.00134 | erreur train: 0.000000 | erreur valid: 0.050000
[  5] perte: 0.00120 | erreur train: 0.000000 | erreur valid: 0.051667
Le temps d'apprentissage: 3.5344624519348145
[  6] perte: 0.00113 | erreur train: 0.000000 | erreur valid: 0.053333
[  7] perte: 0.00108 | erreur train: 0.000000 | erreur valid: 0.053333
[  8] perte: 0.00100 | erreur train: 0.000000 | erreur valid: 0.051667
[  9] perte: 0.00094 | erreur train: 0.000000 | erreur valid: 0.050000
[ 10] perte: 0.00087 | erreur train: 0.000000 | erreur valid: 0.050000
Le temps d'apprentissage: 6.932058811187744
[ 11] perte: 0.00085 | erreur train: 0.000000 | erreur valid: 0.050000
=== Optimisation terminée ===
Le temps d'apprentissage complet: 7.458845138549805
Early s

In [0]:
from sklearn.metrics import accuracy_score
train_pred = R.prediction(train_x_prime)
test_pred = R.prediction(test_x_prime)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

Précision train: 0.99
Précision test : 0.9595714285714285
